In [ ]:
# install required packages

! pip install -qU langchain-community pymupdf
! pip install -qU langchain_huggingface
! pip install -qU sentence-transformers
! pip install -qU langchain-perplexity
! pip install -qU faiss-cpu
! pip install gradio

In [1]:
# import libraries

import os, glob
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader, CSVLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from sentence_transformers import CrossEncoder, util
from langchain_core.prompts import ChatPromptTemplate
from langchain_perplexity import ChatPerplexity
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
import gradio as gr
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

/Users/RajivGaba/aiml_projects/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# import key from local env

import dotenv
dotenv.load_dotenv()
os.environ['PERPLEXITY_API_KEY'] = os.getenv('PPLX_API_KEY')

In [26]:
# Set the key for LLM API : when using google colab
# os.environ['PREPLEXITY_API_KEY'] = userdata.get('PREPLEXITY_API_KEY')

In [5]:
# Set base configuration values

config = {
    'data_path' : '/Users/RajivGaba/aiml_projects/Semantic Spotter/Data/',
    'chunk_size' : 512,
    'chunk_overlap' : 80,
    'vector_store_name' : "faiss_vector",
    'embedding_model' : 'all-MiniLM-L6-v2',
    'refresh_vector_store' : 'N',
    'PPLX_API_KEY' : os.environ['PERPLEXITY_API_KEY'],
    'domain' : 'fashion',
    'chat_model' : "sonar-pro",
    'rerank_model' : 'BAAI/bge-reranker-base'
}

In [12]:
# Download dataset from kaggle using kaggle API

api = KaggleApi()
api.authenticate()
# api.dataset_download_files('promptcloud/myntra-e-commerce-product-data-november-2023', path='./data/', unzip=True)
api.dataset_download_files("ronakbokaria/myntra-products-dataset", path='./data/', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/ronakbokaria/myntra-products-dataset


In [6]:
# Define reusable functions

def get_data_chunks(folder_path):
  """
  This function will create chunks from the files present in the dataset.
  This takes directory where the files exists. Basis the type of file i.e.
  PDF, CSV or text, a loader is initialised and chunks are created from
  content of the data
  """
  # loader = PyMuPDFLoader(pdf_file)
  # documents = loader.load()

  all_documents = []

  # Loop through all files in the folder
  for file_path in folder_path.iterdir():
      if file_path.suffix.lower() == ".pdf":
          loader = PyMuPDFLoader(str(file_path))
      elif file_path.suffix.lower() == ".csv":
          loader = CSVLoader(str(file_path))
      elif file_path.suffix.lower() == ".txt":
          loader = TextLoader(str(file_path))
      else:
          continue  # Skip unsupported file types

      # Load and append documents
      documents = loader.load()
      all_documents.extend(documents)

  # chunking/splitting
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=config['chunk_size'],
      chunk_overlap=config['chunk_overlap'],
      strip_whitespace=True,
      separators=["\n\n", "\n", " ", ""]
  )
  text_chunks = text_splitter.split_documents(documents=all_documents)
  return text_chunks

def get_embeddings_model():
  embedding_model = HuggingFaceEmbeddings(
      model_name=config['embedding_model'],
      show_progress=True,
      multi_process=False,
      model_kwargs={'device': 'mps'}
  )
  return embedding_model

def create_vector_store(text_chunks, embedding_model):
  if config['refresh_vector_store'] == 'Y' or not os.path.exists(config['vector_store_name']):
      vector_store = FAISS.from_documents(text_chunks, embedding_model)
      vector_store.save_local(config['vector_store_name'])
  else:
      vector_store = FAISS.load_local(config['vector_store_name'], embedding_model, allow_dangerous_deserialization=True)
  return vector_store

def get_cross_encoder_score(query, results):
  cross_encoder = CrossEncoder(config['rerank_model'])
  for i, res in enumerate(results):
      ce_score = cross_encoder.predict([query, res.page_content])
      print(ce_score)

def create_chat_client():
  return ChatPerplexity(
      temperature=0,
      pplx_api_key=config['PPLX_API_KEY'],
      model=config['chat_model']
  )

def get_retriever(top_k=5):
  retriever = vector_store.as_retriever(search_kwargs={'k': top_k})
  return retriever

def get_llm_response(query, results, domain):
  system = """
  You are a helpful assistant in {domain} domain.
  Using the information contained in the context, give a comprehensive answer to the question.
  Respond only to the question asked, response should be concise and relevant to the question.
  Provide the number of the source document when relevant.
  If the answer cannot be deduced from the context, do not give an answer.
  If there are image files in the given context, you show the images to the user.

  #####
  Here is the context: {context}
  #####

  #####
  Here is chain of thought:
  User: dresses for beach vacation
  Assistant: Sure. Beach vacations are all about being free and close to nature. Here are suggestions: floral yellow shirt from brand X <image>, midi dress with fine blue prints from brand Y <image>
  User: how about dresses for women in yellow
  Assistant: Sure, here you go: midi dress from brand Y <image>, long floral gown from brand Z <image>
  #####

  You ask the follow up question to the user for relevant pairing suggestions like hat for beach, mufler for cold weather etc.
  """
  human = "{query}"

  prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

  chat = create_chat_client()

  chain = prompt | chat
  response = chain.invoke(
      {
          "context" : results,
          "domain" : config['domain'],
          "query": query,
      }
  )
  return response.content

def get_reranked_query_results(query):
  model = HuggingFaceCrossEncoder(model_name=config['rerank_model'])

  compressor = CrossEncoderReranker(model=model, top_n=5)
  
  compression_retriever = ContextualCompressionRetriever(
      base_compressor=compressor,
      base_retriever=get_retriever()
  )

  compressed_docs = compression_retriever.invoke(query)

  return compressed_docs

def rag_pipeline(user_question):

  # Retrieve relevant documents from the vector store based on the user's question.
  retrieved_documents = get_reranked_query_results(user_question)

  # Extract the page content from the retrieved documents and join them into a single context string.
  formatted_context = "\n\n".join(doc.page_content for doc in retrieved_documents)

  # Call the large language model with the user's question and the formatted context.
  answer = get_llm_response(user_question, formatted_context, config['domain'])
  return answer

def get_answer(user_query, history):
  # return the answer to user query. This function is useful in interaction with Gradio UI
  return rag_pipeline(user_query)

In [7]:

chunked_data = []
folder_path = Path(config['data_path'])
# folder_path = Path("/content/data/")

# Step 1: Generate chunks from the dataset
if config['refresh_vector_store'] == 'Y':
    chunked_data = get_data_chunks(folder_path)


In [8]:
# Step 2: Create embeddings and put them into a vector store
embedding_model = get_embeddings_model()
vector_store = create_vector_store(chunked_data, embedding_model)

In [9]:
query = "party dress for women"
results = get_reranked_query_results(query)


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.97it/s]


In [34]:
get_llm_response(query, results, config['domain'])

'For a **party dress for women**, top options include cocktail dresses, sequin dresses, and elegant gowns in a variety of styles, colors, and lengths. Popular choices feature bold colors, shimmering fabrics, and unique details like ruffles, lace, or asymmetrical cuts[1][2][3][5][6].\n\nKey styles to consider:\n- **Cocktail dresses**: Typically knee-length or midi, made from luxe fabrics like silk, satin, or sequins. These are versatile for most parties and semi-formal events[1][2][4][6].\n- **Sequin and embellished dresses**: Perfect for evening parties, these add glamour and stand out in photos[1][4][6].\n- **Long gowns or maxi dresses**: Ideal for formal parties or evening events, especially in black, metallic, or jewel tones[1][3][5].\n- **Trendy cuts**: Off-shoulder, halter neck, and wrap styles are popular for a modern look[1][5][6].\n- **Bold colors and prints**: Red, emerald, gold, and classic black are always in style, but bright hues and patterns are trending for daytime or su

In [36]:
get_answer("party dresses for women","")

Batches: 100%|██████████| 1/1 [00:00<00:00, 16.67it/s]


'Here are some popular options for **party dresses for women**:\n\n- **Sequin dresses** and **satin dresses** are trending for evening events, offering a glamorous and eye-catching look[2][6].\n- **Little black dresses** remain a classic choice for cocktail parties, while bold colors like red, green, and yellow are great for making a statement[1][2][3][4].\n- **Styles** range from mini and midi lengths to long gowns, with options like off-the-shoulder, halter, and sleeveless cuts[3][4][5].\n- **Unique details** such as ruffles, asymmetrical designs, and embellishments like beading or lace add elegance and personality to your outfit[1][5].\n- **Jumpsuits** in satin or bold colors are also a chic alternative for party wear[2].\n\nWould you like suggestions for pairing accessories, such as statement earrings or a clutch, to complete your party look?'

In [37]:
iface = gr.ChatInterface(
    get_answer,
    type = 'messages',
    chatbot=gr.Chatbot(height=500),
    textbox=gr.Textbox(placeholder="Help me find a formal shirt..", container=False, scale=7),
    title="ClosetAI - Fashion Studio",
    description="Ask ClosetAI anything about fashion products on Mytra",
    theme="glass",
    # examples=["Blazers for men", "Party dresses for women", "Athleisure" , "Sports attire" ],
    cache_examples=True,
)
iface.launch(share=True, debug=False)

/var/folders/w2/6035x_495mg87nyn6pl05mp40000gq/T/ipykernel_78882/4142205462.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=500),
/Users/RajivGaba/aiml_projects/.venv/lib/python3.13/site-packages/gradio/chat_interface.py:323: UserWarning: The type of the gr.Chatbot does not match the type of the gr.ChatInterface.The type of the gr.ChatInterface, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


KeyboardInterrupt: 

In [27]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import Runnable
from langchain_core.messages import HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain_perplexity import ChatPerplexity

# Initialize the ChatPerplexity model
llm = ChatPerplexity(model="sonar-pro", temperature=0)

# Define a prompt template with memory placeholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Set up memory to store conversation history
memory = ConversationBufferMemory(return_messages=True)

# Create the chain with memory
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

# Chat loop
print("Chatbot: Hello! Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye!")
        break

    response = chain.invoke({"input": user_input})
    print(response)


Chatbot: Hello! Type 'exit' to quit.
{'input': 'what is the currency of india', 'history': [HumanMessage(content='what is the currency of india', additional_kwargs={}, response_metadata={}), AIMessage(content='The currency of India is the **Indian rupee**, with the symbol **₹** and the currency code **INR**[1][2][3][4][5]. \n\nThe Indian rupee is subdivided into 100 **paise** (singular: paisa)[1][4]. The Reserve Bank of India is responsible for issuing and regulating the currency[1]. Indian banknotes are commonly found in denominations of 5, 10, 20, 50, 100, 200, 500, and 2000 rupees, while coins are available in 1, 2, 5, and 10 rupees[2]. \n\nThe rupee has a long history, first introduced in its modern form in 1540, and is one of the oldest continuously used currencies in the world[3].', additional_kwargs={}, response_metadata={})], 'text': 'The currency of India is the **Indian rupee**, with the symbol **₹** and the currency code **INR**[1][2][3][4][5]. \n\nThe Indian rupee is subdiv